# Gerador de Data-Schemas - Projeto GEI

Este notebook gera automaticamente a documentação de schema para todas as tabelas do projeto GEI.

**Para cada tabela, serão gerados 2 arquivos:**
1. `{schema}__{tabela}__describe.txt` → DESCRIBE FORMATTED
2. `{schema}__{tabela}__sample.txt` → SELECT * LIMIT 10

---

## 📋 Total de Tabelas:
- **9 tabelas originais** (fontes externas)
- **17 tabelas intermediárias** (processadas pelo GEI)
- **Total: 26 tabelas** → 52 arquivos gerados

## 1️⃣ Configuração do Ambiente

In [ ]:
import sys
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/poc")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/plugins")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

# Import libs python
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime

# Import libs internas
from utils import spark_utils_session as utils
from hooks.hdfs.hdfs_helper import HdfsHelper
from jobs.job_base_config import BaseETLJobClass

import poc_helper
poc_helper.load_env("PROD")

print("✅ Ambiente configurado com sucesso!")

## 2️⃣ Inicializar Sessão Spark

In [ ]:
def get_session(profile: str, dynamic_allocation_enabled: bool = True) -> utils.DBASparkAppSession:
    """Gera DBASparkAppSession."""
    
    app_name = "gei_data_schema_generator"
    
    spark_builder = (utils.DBASparkAppSession
                     .builder
                     .setAppName(app_name)
                     .usingProcessProfile(profile)
                    )
    
    if dynamic_allocation_enabled:
        spark_builder.autoResourceManagement()

    return spark_builder.build()

session = get_session(profile='efd_t2')
spark = session.sparkSession

print(f"✅ Sessão Spark ('{spark.sparkContext.appName}') ativa e pronta para uso.")

## 3️⃣ Definição das Tabelas

In [ ]:
from pathlib import Path

# =============================================================================
# TABELAS ORIGINAIS (Fontes de Dados)
# =============================================================================

TABELAS_ORIGINAIS = [
    ("usr_sat_ods", "vw_ods_contrib", "Dados cadastrais de contribuintes (ODS)"),
    ("usr_sat_ods", "vw_cad_vinculo", "Vínculos cadastrais (sócios/responsáveis)"),
    ("usr_sat_ods", "sna_pgdasd_estabelecimento_raw", "Dados brutos PGDAS-D"),
    ("nfe", "nfe", "Notas Fiscais Eletrônicas"),
    ("c115", "c115_dados_cadastrais_dest", "Convênio 115"),
    ("usr_sat_fsn", "fsn_conta_bancaria", "Contas bancárias"),
    ("rais_caged", "vw_rais_vinculos", "Vínculos empregatícios RAIS/CAGED"),
    ("usr_sat_admcc", "acc_r66_totalestab", "Meios de pagamento"),
    ("neaf", "empresa_indicio", "Indícios fiscais NEAF"),
]

# =============================================================================
# TABELAS INTERMEDIÁRIAS (Tabelas GEI)
# =============================================================================

TABELAS_INTERMEDIARIAS = [
    # Principais
    ("gessimples", "gei_percent", "Tabela principal com scores e níveis de risco"),
    ("gessimples", "gei_cnpj", "Relação CNPJ ↔ Grupo Econômico"),
    ("gessimples", "gei_cadastro", "Dados cadastrais consolidados"),
    ("gessimples", "gei_contador", "Contadores dos grupos"),
    ("gessimples", "gei_socios_compartilhados", "Sócios em múltiplas empresas"),
    ("gessimples", "gei_c115_ranking_risco_grupo_economico", "Ranking de risco C115"),
    ("gessimples", "gei_funcionarios_metricas_grupo", "Métricas RAIS/CAGED"),
    ("gessimples", "gei_pagamentos_metricas_grupo", "Métricas de meios de pagamento"),
    ("gessimples", "gei_c115_metricas_grupos", "Métricas C115 adicionais"),
    ("gessimples", "gei_ccs_metricas_grupo", "Métricas de contas compartilhadas"),
    ("gessimples", "gei_ccs_ranking_risco", "Ranking de risco CCS"),

    # Detalhadas CCS
    ("gessimples", "gei_ccs_cpf_compartilhado", "CPFs com contas em múltiplos CNPJs"),
    ("gessimples", "gei_ccs_sobreposicao_responsaveis", "Responsáveis com períodos sobrepostos"),
    ("gessimples", "gei_ccs_padroes_coordenados", "Eventos coordenados"),

    # Inconsistências
    ("gessimples", "gei_indicios", "Indícios fiscais catalogados"),
    ("gessimples", "gei_nfe_completo", "NFe com inconsistências detectadas"),
    ("gessimples", "gei_pgdas", "Dados PGDAS mensais"),
]

print(f"📊 Total de tabelas configuradas:")
print(f"   - Originais: {len(TABELAS_ORIGINAIS)}")
print(f"   - Intermediárias: {len(TABELAS_INTERMEDIARIAS)}")
print(f"   - TOTAL: {len(TABELAS_ORIGINAIS) + len(TABELAS_INTERMEDIARIAS)}")

## 4️⃣ Funções de Processamento

In [ ]:
def criar_diretorios():
    """Cria estrutura de diretórios para os data-schemas."""
    base_dir = Path("data-schemas")
    originais_dir = base_dir / "originais"
    intermediarias_dir = base_dir / "intermediarias"

    originais_dir.mkdir(parents=True, exist_ok=True)
    intermediarias_dir.mkdir(parents=True, exist_ok=True)

    print(f"✅ Diretórios criados:")
    print(f"   - {originais_dir}")
    print(f"   - {intermediarias_dir}")

    return originais_dir, intermediarias_dir


def salvar_resultado(conteudo: str, caminho: Path):
    """Salva o resultado em um arquivo."""
    with open(caminho, 'w', encoding='utf-8') as f:
        f.write(conteudo)
    print(f"   ✅ Salvo: {caminho}")


def processar_tabela(spark, schema: str, tabela: str, descricao: str, diretorio: Path):
    """
    Processa uma tabela: executa DESCRIBE FORMATTED e SELECT LIMIT 10.
    Salva os resultados em arquivos separados.
    """
    tabela_completa = f"{schema}.{tabela}"
    print(f"\n{'='*80}")
    print(f"🔄 Processando: {tabela_completa}")
    print(f"   Descrição: {descricao}")
    print(f"{'='*80}")

    # Nome base do arquivo
    nome_arquivo_base = f"{schema}__{tabela}"

    # =========================================================================
    # 1. DESCRIBE FORMATTED
    # =========================================================================
    try:
        print(f"\n📋 Executando DESCRIBE FORMATTED {tabela_completa}...")
        describe_df = spark.sql(f"DESCRIBE FORMATTED {tabela_completa}")

        # Converte para string formatada
        describe_output = []
        describe_output.append(f"# DESCRIBE FORMATTED: {tabela_completa}\n")
        describe_output.append(f"# Descrição: {descricao}\n")
        describe_output.append(f"# Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        describe_output.append("\n" + "="*80 + "\n\n")

        # Coleta os dados
        rows = describe_df.collect()
        for row in rows:
            linha = " | ".join([str(col) if col is not None else "" for col in row])
            describe_output.append(linha + "\n")

        # Salva o DESCRIBE FORMATTED
        describe_path = diretorio / f"{nome_arquivo_base}__describe.txt"
        salvar_resultado(''.join(describe_output), describe_path)

    except Exception as e:
        print(f"   ❌ ERRO ao executar DESCRIBE FORMATTED: {e}")
        describe_output = [f"ERRO: {e}\n"]
        describe_path = diretorio / f"{nome_arquivo_base}__describe.txt"
        salvar_resultado(''.join(describe_output), describe_path)

    # =========================================================================
    # 2. SELECT * LIMIT 10
    # =========================================================================
    try:
        print(f"\n📊 Executando SELECT * FROM {tabela_completa} LIMIT 10...")
        select_df = spark.sql(f"SELECT * FROM {tabela_completa} LIMIT 10")

        # Converte para string formatada
        select_output = []
        select_output.append(f"# SELECT * FROM {tabela_completa} LIMIT 10\n")
        select_output.append(f"# Descrição: {descricao}\n")
        select_output.append(f"# Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        select_output.append("\n" + "="*80 + "\n\n")

        # Adiciona o schema
        select_output.append("## SCHEMA:\n\n")
        for field in select_df.schema.fields:
            select_output.append(f"{field.name} | {field.dataType} | {field.nullable}\n")

        select_output.append("\n" + "="*80 + "\n\n")
        select_output.append("## DADOS (primeiras 10 linhas):\n\n")

        # Coleta os dados em formato string
        select_output.append(select_df._jdf.showString(10, 20, False))

        # Salva o SELECT
        select_path = diretorio / f"{nome_arquivo_base}__sample.txt"
        salvar_resultado(''.join(select_output), select_path)

    except Exception as e:
        print(f"   ❌ ERRO ao executar SELECT: {e}")
        select_output = [f"ERRO: {e}\n"]
        select_path = diretorio / f"{nome_arquivo_base}__sample.txt"
        salvar_resultado(''.join(select_output), select_path)

    print(f"\n✅ Tabela {tabela_completa} processada com sucesso!")

print("✅ Funções de processamento definidas!")

## 5️⃣ EXECUTAR - Processar Todas as Tabelas

**⚠️ ATENÇÃO:** Esta célula irá processar todas as 26 tabelas. O tempo estimado é de **5-10 minutos** dependendo da performance do banco.

In [ ]:
print("\n" + "="*80)
print("GERADOR DE DATA-SCHEMAS - PROJETO GEI")
print("="*80)
print(f"Início: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total de tabelas a processar: {len(TABELAS_ORIGINAIS) + len(TABELAS_INTERMEDIARIAS)}")
print("="*80 + "\n")

# Cria estrutura de diretórios
dir_originais, dir_intermediarias = criar_diretorios()

# Contadores
total_sucesso = 0
total_erro = 0

# =============================================================================
# Processar TABELAS ORIGINAIS
# =============================================================================
print("\n" + "🔵"*40)
print("PROCESSANDO TABELAS ORIGINAIS (Fontes de Dados)")
print("🔵"*40 + "\n")

for schema, tabela, descricao in TABELAS_ORIGINAIS:
    try:
        processar_tabela(spark, schema, tabela, descricao, dir_originais)
        total_sucesso += 1
    except Exception as e:
        print(f"❌ ERRO CRÍTICO ao processar {schema}.{tabela}: {e}")
        total_erro += 1

# =============================================================================
# Processar TABELAS INTERMEDIÁRIAS
# =============================================================================
print("\n" + "🟢"*40)
print("PROCESSANDO TABELAS INTERMEDIÁRIAS (Tabelas GEI)")
print("🟢"*40 + "\n")

for schema, tabela, descricao in TABELAS_INTERMEDIARIAS:
    try:
        processar_tabela(spark, schema, tabela, descricao, dir_intermediarias)
        total_sucesso += 1
    except Exception as e:
        print(f"❌ ERRO CRÍTICO ao processar {schema}.{tabela}: {e}")
        total_erro += 1

# =============================================================================
# RELATÓRIO FINAL
# =============================================================================
print("\n" + "="*80)
print("RELATÓRIO FINAL")
print("="*80)
print(f"✅ Tabelas processadas com sucesso: {total_sucesso}")
print(f"❌ Tabelas com erro: {total_erro}")
print(f"📁 Arquivos salvos em: data-schemas/")
print(f"   - Originais: {len(TABELAS_ORIGINAIS) * 2} arquivos")
print(f"   - Intermediárias: {len(TABELAS_INTERMEDIARIAS) * 2} arquivos")
print(f"   - Total: {(len(TABELAS_ORIGINAIS) + len(TABELAS_INTERMEDIARIAS)) * 2} arquivos")
print(f"\nFim: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80 + "\n")

## 6️⃣ Verificar Resultados

In [ ]:
import os

# Lista todos os arquivos gerados
print("📂 Arquivos gerados:\n")

for root, dirs, files in os.walk("data-schemas"):
    level = root.replace("data-schemas", "").count(os.sep)
    indent = " " * 2 * level
    print(f"{indent}📁 {os.path.basename(root)}/")
    subindent = " " * 2 * (level + 1)
    for file in sorted(files):
        size = os.path.getsize(os.path.join(root, file))
        print(f"{subindent}📄 {file} ({size:,} bytes)")

## 7️⃣ (OPCIONAL) Processar Tabela Individual

Use esta célula se quiser processar apenas uma tabela específica:

In [ ]:
# Configurar aqui
SCHEMA = "gessimples"
TABELA = "gei_percent"
DESCRICAO = "Tabela principal com scores"

# Processar
output_dir = Path("data-schemas/teste")
output_dir.mkdir(parents=True, exist_ok=True)

processar_tabela(spark, SCHEMA, TABELA, DESCRICAO, output_dir)